In [1]:
from __future__ import print_function

from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt

# %matplotlib inline

import json
import pickle
import os

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
from faker import Faker
import random
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [ ]:
def prepare_dataset(m, Tx = 42, Ty = 42):
    dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)
    X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)
    return X, Y, Xoh, Yoh

prepare_dataset(31767, 42, 42)

100%|██████████| 31767/31767 [00:00<00:00, 540198.32it/s]

['ZNVLLVLTCMBVAHFD', 'GAAYYGLVJYXZFDCYOKPYNYNQEK', 'INQDXROYYCXFXAKFI', 'UBHUMZRPYQKFKHGQUJCKCIJFWXWZKBZXYQ', 'KELKRYLYVSWZBZFGLLGGYHDNULJRQ', 'YMPFWZLEVFTMBJKBUEDROCXQIDKIVNR', 'NLJKEYYHWMMEXIPOQEYBCKB', 'JFPAWJVDCVEVDPJVERCQCBPEUDOEUOOY', 'JFSHZLLASNTODXGXXEU', 'ZNXTBDUTBTBYKDXVMSKILVOLJKJZKBB', 'QGXYAZSPZZEBQTPFU', 'LAHBWZTVZVYQQZSNLKSITYOMEDHQCBXOG', 'NCBQCXOLPCXVDVHNVGIONUOIEIH', 'JELOXYHWCAXZUURKYRPMDOGMHXJVVZ', 'WHWYWALNWMFOXBMKYGDEFO', 'IGQOCMOACGGOREXVUGAIOBQAUZSQMWRKS', 'REJIMZOCVKTZBQFAJTCSFIEJI', 'MZHHGANSQMSGMBFEEGC', 'UNHECFAXCOXHBXFYTKUCC', 'HGVKYQODZMDEUWJBUQNZACXQVJOHNWED', 'OMSAEZRGHQJOKXKBUECGFXHLJX', 'GMCWCMHBTTRBFOHLIFUQYOJEWYJKSEGNC', 'AMJLGKQBNVEHDUJDFNUQTMHJQWRHFZXRE', 'ZAHDXZLJKTJFQQYKVQCVHRCNMRKGFQNRDL', 'RAMKYFWHHOBMUUWAHLPYSCGLKDSPQIDCML', 'HMHHRYSVZSFMQCPGDNFETUXJAKL', 'KZJDLWLNGTUOUJQQKKPHZPBQEBVR', 'OEXOLDOGHYKVHOQAERJKTNOHFXDKKUQZ', 'UFPDCMHDKTFOFEVCDLPYECXWXRYHJKJKUM', 'YFLOLDLVCYTVCULAERJJHYXOUOWHVCQXCL', 'UNBBOQODZMZECUSAEQUI', 'NLJATKSJZHJAAOOKLFNYNY

In [3]:
print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (31767, 42)
Y.shape: (31767, 42)
Xoh.shape: (31767, 42, 28)
Yoh.shape: (31767, 42, 28)


In [4]:
n_a = 128
n_values = 28
n_s = 128

In [5]:
reshapor = Reshape((1, 28))                        # Used in Step 2.B of djmodel(), below
LSTM_cell = LSTM(n_a, return_state = True)         # Used in Step 2.C
densor = Dense(n_values, activation='softmax')   

In [6]:
def djmodel(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the music data 
    
    Returns:
    model -- a keras model with the 
    """
    
    # Define the input of your model with a shape 
    X = Input(shape=(Tx, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    ### START CODE HERE ### 
    # Step 1: Create empty list to append the outputs while you iterate (≈1 line)
    outputs = []
    
    # Step 2: Loop
    for t in range(Tx):
        
        # Step 2.A: select the "t"th time step vector from X. 
        x = Lambda(lambda x: X[:,t,:])(X)
        # Step 2.B: Use reshapor to reshape x to be (1, n_values) (≈1 line)
        x = reshapor(x)
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
        out = densor(a)
        # Step 2.E: add the output to "outputs"
        outputs.append(out)
        
    # Step 3: Create model instance
    model = Model([X, a0, c0], outputs)
    
    ### END CODE HERE ###
    
    return model

In [7]:
model = djmodel(Tx = 42 , n_a = 128, n_values = 28)

# model = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0, 1))


Instructions for updating:
Colocations handled automatically by placer.


In [8]:
history = model.fit([Xoh, s0, c0], outputs, epochs=10, batch_size=100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
31767/31767 [==============================] - 43s 1ms/step - loss: 69.7780 - dense_1_loss: 0.0342 - dense_1_acc: 0.1840 - dense_1_acc_1: 0.3659 - dense_1_acc_2: 0.1746 - dense_1_acc_3: 0.2161 - dense_1_acc_4: 0.3552 - dense_1_acc_5: 0.1167 - dense_1_acc_6: 0.2698 - dense_1_acc_7: 0.2384 - dense_1_acc_8: 0.1917 - dense_1_acc_9: 0.1629 - dense_1_acc_10: 0.1783 - dense_1_acc_11: 0.2793 - dense_1_acc_12: 0.1887 - dense_1_acc_13: 0.3006 - dense_1_acc_14: 0.3218 - dense_1_acc_15: 0.2997 - dense_1_acc_16: 0.3206 - dense_1_acc_17: 0.2787 - dense_1_acc_18: 0.3173 - dense_1_acc_19: 0.4122 - dense_1_acc_20: 0.3895 - dense_1_acc_21: 0.4762 - dense_1_acc_22: 0.5090 - dense_1_acc_23: 0.4838 - dense_1_acc_24: 0.5323 - dense_1_acc_25: 0.5192 - dense_1_acc_26: 0.6043 - dense_1_acc_27: 0.6157 - dense_1_acc_28: 0.6373 - dense_1_acc_29: 0.7106 - dense_1_acc_30: 0.7686 - dense_1_acc_31: 0.7992 - dense_1_acc_32: 0.8699 - dense_1_acc_33: 0.9239 - de

31767/31767 [==============================] - 22s 678us/step - loss: 23.0935 - dense_1_loss: 4.0269e-07 - dense_1_acc: 0.7041 - dense_1_acc_1: 0.7771 - dense_1_acc_2: 0.7680 - dense_1_acc_3: 0.8628 - dense_1_acc_4: 0.9001 - dense_1_acc_5: 0.8323 - dense_1_acc_6: 0.8603 - dense_1_acc_7: 0.8667 - dense_1_acc_8: 0.8818 - dense_1_acc_9: 0.8804 - dense_1_acc_10: 0.8600 - dense_1_acc_11: 0.8716 - dense_1_acc_12: 0.8662 - dense_1_acc_13: 0.8816 - dense_1_acc_14: 0.8698 - dense_1_acc_15: 0.8352 - dense_1_acc_16: 0.8662 - dense_1_acc_17: 0.8443 - dense_1_acc_18: 0.7950 - dense_1_acc_19: 0.8362 - dense_1_acc_20: 0.8438 - dense_1_acc_21: 0.8434 - dense_1_acc_22: 0.7942 - dense_1_acc_23: 0.7929 - dense_1_acc_24: 0.7786 - dense_1_acc_25: 0.8453 - dense_1_acc_26: 0.8165 - dense_1_acc_27: 0.8364 - dense_1_acc_28: 0.7746 - dense_1_acc_29: 0.8193 - dense_1_acc_30: 0.8491 - dense_1_acc_31: 0.8648 - dense_1_acc_32: 0.9261 - dense_1_acc_33: 0.9449 - dense_1_acc_34: 0.9759 - dense_1_acc_35: 0.9946 - dense

31767/31767 [==============================] - 22s 682us/step - loss: 18.3146 - dense_1_loss: 3.7488e-07 - dense_1_acc: 0.7936 - dense_1_acc_1: 0.8050 - dense_1_acc_2: 0.8233 - dense_1_acc_3: 0.8983 - dense_1_acc_4: 0.9261 - dense_1_acc_5: 0.8737 - dense_1_acc_6: 0.9161 - dense_1_acc_7: 0.9256 - dense_1_acc_8: 0.9164 - dense_1_acc_9: 0.9113 - dense_1_acc_10: 0.9245 - dense_1_acc_11: 0.9292 - dense_1_acc_12: 0.9244 - dense_1_acc_13: 0.9139 - dense_1_acc_14: 0.9239 - dense_1_acc_15: 0.8834 - dense_1_acc_16: 0.9100 - dense_1_acc_17: 0.8932 - dense_1_acc_18: 0.8482 - dense_1_acc_19: 0.8898 - dense_1_acc_20: 0.8741 - dense_1_acc_21: 0.8736 - dense_1_acc_22: 0.8385 - dense_1_acc_23: 0.8516 - dense_1_acc_24: 0.8147 - dense_1_acc_25: 0.8786 - dense_1_acc_26: 0.8511 - dense_1_acc_27: 0.8607 - dense_1_acc_28: 0.8046 - dense_1_acc_29: 0.8479 - dense_1_acc_30: 0.8696 - dense_1_acc_31: 0.8793 - dense_1_acc_32: 0.9346 - dense_1_acc_33: 0.9462 - dense_1_acc_34: 0.9759 - dense_1_acc_35: 0.9946 - dense

31767/31767 [==============================] - 22s 679us/step - loss: 15.1910 - dense_1_loss: 5.1772e-07 - dense_1_acc: 0.8652 - dense_1_acc_1: 0.8269 - dense_1_acc_2: 0.8701 - dense_1_acc_3: 0.9144 - dense_1_acc_4: 0.9446 - dense_1_acc_5: 0.9032 - dense_1_acc_6: 0.9577 - dense_1_acc_7: 0.9572 - dense_1_acc_8: 0.9426 - dense_1_acc_9: 0.9445 - dense_1_acc_10: 0.9535 - dense_1_acc_11: 0.9457 - dense_1_acc_12: 0.9578 - dense_1_acc_13: 0.9341 - dense_1_acc_14: 0.9464 - dense_1_acc_15: 0.9252 - dense_1_acc_16: 0.9438 - dense_1_acc_17: 0.9208 - dense_1_acc_18: 0.8816 - dense_1_acc_19: 0.9151 - dense_1_acc_20: 0.8976 - dense_1_acc_21: 0.8974 - dense_1_acc_22: 0.8778 - dense_1_acc_23: 0.8885 - dense_1_acc_24: 0.8494 - dense_1_acc_25: 0.8892 - dense_1_acc_26: 0.8748 - dense_1_acc_27: 0.8797 - dense_1_acc_28: 0.8309 - dense_1_acc_29: 0.8830 - dense_1_acc_30: 0.8779 - dense_1_acc_31: 0.8873 - dense_1_acc_32: 0.9419 - dense_1_acc_33: 0.9496 - dense_1_acc_34: 0.9757 - dense_1_acc_35: 0.9946 - dense

In [12]:
test_cipher = []
test_plain = []

In [13]:
with open('test.txt', 'r', encoding = 'utf-8') as f:
    test_lines = f.read().split('\n')

In [14]:
for line in test_lines:
#     print(line.split('\t'))
    input_text, target_text = line.split('\t')
    test_cipher.append(input_text)
    test_plain.append(target_text)


In [15]:
predicts = []

In [16]:
for i in range(len(test_cipher)):
    source = string_to_int(test_cipher[i], Tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source)))
    source = source.reshape((1, source.shape[0], source.shape[1]))

    prediction = model.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis=-1)
    output = []
    for k in prediction:
        if k == 27:
            break
        output.append(inv_machine_vocab[int(k)])
    predicts.append(''.join(output))
    print("source:", test_cipher[i])
    print("actual:", test_plain[i])
    print("output:", ''.join(output), "\n")

source: OGBDCFDCCVVOMMJDLKOJCBP
actual: PROCESSLEADERPERSONWEAR
output: PROCESSLEADERPERSONWEAR 

source: VFEAAZTVZMEMUQYAJACZOQEFXBG
actual: CIVILEVERYTHINGORNEXTSOURCE
output: CIAILEVERYTHINGORNEPTSOURCE 

source: QFYLOMLHGQFZUEQOSHLSIYOLXDS
actual: MIGHTPROFESSIONALTURNNEARLY
output: MIGHTPRORESSIONALTURNNEARLY 

source: NSFAOVCPFHJOMEHGUURIECJAKO
actual: ADMITIMAGINEROLEESPECIALLY
output: ALMITIMAGINEROLEESPECIALLT 

source: UEFKAZJBOOKBC
actual: SAMELEFTSTART
output: SAIELEFTSTART 

source: ZMDIMGBCQMCCFDOFFUKII
actual: TOWARDPLAYMUSICLISTEN
output: TORARDTLAWMUSICLISTEN 

source: DAUKYAYVMNUGTOYATLPYYOGBJYHRFANQBMY
actual: RECENTNEWSRACEGOVERNMENTCOLLEGEFORM
output: RECENTNEWSRACEGOVERNNENTROLLETETORS 

source: QAVHCQVGSSUOATOGJSUIIEXXEZE
actual: MEELECTIONREDUCERULENOTHING
output: MEELECTIONREDUCERULENOTHING 

source: WHMTMPOBVTJEVWJFUNMHJEPCIIOEFCGGC
actual: INFORMATIONONCELEASTWORKERTHENOLD
output: INOORMATIONONCELEASTHORSERTGENAOD 

source: GAHCBKSVZNEGVIRKYMCTNVFBNITWBYGJQ
a

source: YFEKXYHWCVUPCOHFLLCU
actual: LIVEHOUSEARTTELLSEEM
output: LIAEHOUSEARTTELLSEEM 

source: BFPCEYCAQGTUQQOGJANOOOG
actual: KINDCOMPARECONCERNOFTEN
output: TINDCOMPARECONCERNONTEN 

source: OABMAZBPZOXUIVKDSJOFIOHJEXXM
actual: PEOPLEPARTICULARLYNONEFRIEND
output: PEOPLEPARTICULARLWNONEFRIENR 

source: QMPKNKABNHJACAKYPEYCCOBNSVGLF
actual: MONEYNOTHINGTHANKLIVEEYEWHERE
output: MONEYNOTHINGTHANKLIEEEAERTERE 

source: PHSUETLVMQGMUVJFXDIIQMEEMXM
actual: ONLYCAREWEWHILELAWYERWONDER
output: ONLCCAREWERHILELAHYERYONDER 

source: PHJTXYHDCJWOAOXOFEVGDKJB
actual: ONTOHOUREDGEDETAILFALLAT
output: ONTOMOUREDGEDETAILCALLAT 

source: OMDKMVVZQPYXVUJDUUKEIVXXXYB
actual: POWERITFALLINTERESTINGTHROW
output: PORERITFALLINTERESTINHTHROC 

source: ZFFKCBOLYPLTDJJGAVCSOKJBIPTZK
actual: TIMEEXACTLYPAGEEXPERTLATEGOAL
output: TIMEEPACTLYPAGEEOPERTLATEIONL 

source: INHOCIWDKGBKBWXOJHVEDW
actual: WHATEVERPROJECTARTFILM
output: WHATEVERPROAECTARTTILM 

source: DAMHCQVLBQKBDVHABGCQNVQMAU
actual: REFLECTCL

source: YAHBYMWDOTJXVUC
actual: LEARNPERSONINTO
output: LEARNRERSONINTO 

source: XHJKMQUPZRTZDKJHUGIFIOQOFJDK
actual: ENTERCHARGESAYEVERYONEIMPACT
output: EATERCHARGESAYETEREONESTPALT 

source: JESHMZORWMJECO
actual: FALLREADMYNOTE
output: FALLREADMWNOTE 

source: WONBTIWLNTBZBUCXYMCSOBQEKOMPWY
actual: IMPROVECHOOSETOWNCERTAINLYRICH
output: IPPROGECHOOSETOWNCERTAINTTRIES 

source: DAHCWZMHHJJXJAXMNVPFZOPBZUOSKW
actual: READSECONDNIGHTBYPROPERTYSTYLE
output: READSECONDNIGHTPYPROPERTYSTYLE 

source: OEXOOJHWBTGXVXXGXWEGQOOJPIGHCSQY
actual: PARTTHUSLOWINSTEADCAREERFREEWISH
output: PARTTHUSLOWINSTEADCAREERRREENITS 

source: YAHBYSXYCBUXCOSVFHTIMQKKJXYY
actual: LEARNFINEWRITERHITKEYSUCCESS
output: LEARNFINEWRITERUITHEWSUCAESS 

source: QFWCAZEGYIXVTVCQULDSDNEJKXYY
actual: MIDDLEWITHINCLOSEEARLYORLESS
output: MIDDLEWITHINCLOSEEARLYORLESS 

source: IFMKSJOBEQUPDDQEMPCSIWOEV
actual: WIFEWHATCERTAINGOVERNMENT
output: WIOEWHATCERTAINGOREREMENT 

source: HGVKYFADYWTZCMJDMVKETYGLVRTV
actual: GREE

source: RAFTEWOBYITHFOHHUUMDCIQUEBSHQ
actual: DEMOCRATTHEMSELVESSPECIFICYET
output: DEMOCRATTHEMSELLESSPECSAICYET 

source: REGLLKSRCNQXCOIKJL
actual: DAYHANDDESPITEFIRE
output: DAYAANDDESPITEFIRE 

source: QEGWOWHLYXUOUBFYZGCGFOOPIZBPQYXR
actual: MAYSTRUCTUREIFINCREASEEVENWITHIN
output: MAYSTRUCTUREIPINCREASEERENCITRIN 

source: WBWKRYMDQOCBCAJDUCDAHDQEI
actual: ITDEMOCRATMRTHEREMAGAZINE
output: IODEMOCRATMRTHEREMADALINE 

source: INBPELLDCSEZQEQTEEKMQO
actual: WHOMCURRENTSOONCULTURE
output: WHOMCARRENTSOONCULTURE 

source: AMXSCVQOYSBPQMBAJHLYNXBVQMGLWYXGC
actual: NORWEIGHTNOTOPPORTUNITYPOWERCHILD
output: EORWEIGHTNOTOPPORTUNITAPORERESIOD 

source: LMCBCTRCTJTAMOJ
actual: YOUREALLYDEGREE
output: SOUREALLYDEGREE 

source: IAAAIJMHHNXYBZOASKPTBWJE
actual: WEHIGHCONSIDERCOLORHUMAN
output: WEEIGHCONSIDEREOLORHUMAN 

source: OGVWCKVBCGCYBXODFQCHCQXQQWGQMW
actual: PRESENTTERMDESCRIBETESTSOMEONE
output: PRESENTTERMDESCRIGETESTSOSEONE 

source: KCCKWMWLVVYUDZSN
actual: BLUESPECIALCARRY
outpu

source: ZGVIOJAJOQCGMZFOQLKSTRYAI
actual: TREATHOUSEMARRIAGETROUBLE
output: TREATHOUSEMARRIASETROUSLE 

source: UBQHAZIABVXVXEMFXJQGMIEODYX
actual: STILLEXPLAINLOWLAYWAYCOMMON
output: STILLEVPLAINLOWLAHLAYCOMLON 

source: YECFXJWDCJBQVMSGVHIONYJEJRVR
actual: LAUGHHEREDOWNPRETTYFINANCIAL
output: LAUGHHEREDOPNPRETTYTINANRIAL 

source: VGVIOZAJYCXVDVHNZKDVAXEM
actual: CREATEOUTFINALLYCOACHTOO
output: CREATEOUTEINALLYCOACHTOO 

source: UBHBULVVUQMCCDUG
actual: STARBUTEXECUTIVE
output: STARLUTERECUTITE 

source: NOBQIFUVNXCGVOFCDLP
actual: AMONGSHEHUMANEITHER
output: APONGSHEHUMANEITHER 

source: OZOHGQCJOHMZURJFXJMFSUEEV
actual: PUBLICMUSICSIZELAYSOFRONT
output: PUTLICMUSICSISELAHSORRONT 

source: LAJSGAUVQOCEVULHMFEI
actual: YETWITHEATMONTHVOICE
output: SETFITHEATMONTHLOICE 

source: DALMTKDVYBBZQTSTUGNVVZOBIIRPMW
actual: RESPONSETWOSOURCEROCKDETERMINE
output: RESPONSETWOSOURCEROCHDETERMINE 

source: HAJCGFMJONXEVAKDH
actual: GETDISCUSSIONHARD
output: NETDISCUSSIONHARD 

source: OMNYATVGS

...

source: GZLRLKSDCVYXCKBAMGXSCOGQIXVFF
actual: HUSBANDREALITYPOORGREENSEEAGE
output: RUSMANDREALITYPOORGREENSEEAEE 

source: RFMGCWWYEQTVNDSAYCCYOHPGJXKWMM
actual: DIFFERENCEENVIRONMENTPRICEFUND
output: DIPFERENCEENVIRONMENTARICETCDD 

source: HELQTAWDQJXEROKQEGCHTEFMKLFVUE
actual: GASNOTERADIOMEASURETOOHOLDKNOW
output: NASNOTERADIOMEASURETOOSOLDEAON 

source: XHEAMYYXCSEGXBFYFUJDHCGBEZEMUZDKHL
actual: ENVIRONMENTALFINISHPAINTINGDOTHERE
output: EAAIRONMENTALEINISYPAINTINGDOLTERE 

source: IALTEVOCFGBCZPKUMGSIHY
actual: WESOCIALGROUPMAJORMEAN
output: WESOCIALGROUPMAEORREAN 

source: QFSHGYYPEGBZFICAJMNYOCGFICTQL
actual: MILLIONACROSSDOORCONTINUEBOOK
output: MILLIONACROSSDOORCONTINUEDOOR 

source: NRHAYFVIZTGPYZJOSKRIQBXGQZAQQW
actual: AGAINSTGROWTHREALOPERATIONVOTE
output: ADAINSTGROWTHREALOSERATIONGOTE 

source: ZNHQEZLBQHJTUOOGTKYVCIOEVSMSHUKD
actual: THANCERTAINPIECEVOICECENTURYSORT
output: THANCERTAINEIECECOICECENTURYEOET 

source: JCGOMTSGYHBVDVFYIKPUHXQMHQPLFXGAAK
actual: FLYTRADIT

source: JGBPSTVLNWXPUPBDMPCUTUOFFYX
actual: FROMWATCHBITIMPROVEMOREUPON
output: FROMFATCHMITIMPROREMOREPUON 

source: OAXWTKOCCCSOTUHOYWZEDKQMH
actual: PERSONALEFFECTLANDBILLION
output: PERSONALEFFECTLANDPILLION 

source: UMFKOJXYFJKCJAXGJHNHHKFMS
actual: SOMETHINGDAUGHTERTOTALHOW
output: SOMETHINGDAUGHTERTOTALHOU 

source: WHJKMIXVMVFTMORKHLOHUN
actual: INTERVIEWASPRESIDENTBY
output: INTERGIEWASPRESIDENTRY 

source: VMSHCDWRVCSOMOQTUUNMOPOJH
actual: COLLEGEDIFFERENCESOUTHERN
output: COLLESEDIEFERENCESOUTHERN 

source: XSYKETCVZVXV
actual: EDGECAMERAIN
output: ELUECALERAIN 

source: AMJAEZBDSLTBCKRAZFDQJBXNXKLLFW
actual: NOTICEPROPERTYSOCIALWATERTHREE
output: EOTICEPROPERTYSOCIALAATERTRREE 

source: VESHUUAHJCXVDVHOBNZQC
actual: CALLBLOODFINALLAWABLE
output: CALLLLOODEINALLAWAPLE 

source: ZNHQETLVKGTNBQXQMMYIONRLBX
actual: THANCAREPREVENTSOCIETYPAGE
output: THANCAREPREVENTSOCIETIPACE 

source: NGBYYGNVYGTGXDDGDFS
actual: AROUNDYETREALIZEHIM
output: AROUNDYETREALIIERIM 

source: UAJMMT

source: XFJLCWWMQFEFHXOKUAKEFXGNTXMYDGVK
actual: EITHEREXACTLYSCIENTISTNEVERSPACE
output: EITHERECACTLYSCIENTISTNEOERSLANE 

source: QKLKASOLYHDXCKKCVLOHNEGQQSOEFQE
actual: MYSELFACTIVITYATTENTIONSOUTHERN
output: MESELFACTIVITYATTENTIONSOUTHERN 

source: ZNXTSAWCBVUODFJNMAASCBDAZGTRVNN
actual: THROWTELLAREABEYONDREALLYPOLICE
output: THROWTELLAREASEYONDREALLYEOLIEE 

source: QEXECAWECGLZIBIGJAYVC
actual: MARKETEVERYSUFFERNICE
output: MARCETECERYSUEFERNICE 

source: QEXECAWPOMEGLAKYHNURT
actual: MARKETEASYTAXHANDALSO
output: MARCETEASYTAFHANDALSO 

source: UAUTYGQDSXJYXHQR
actual: SECONDGROUNDLAND
output: SECONDGROUNDLAND 

source: PIMAEZLBQWYOZERKVFFIQBXXIIKLUD
actual: OFFICERTABLEPOSITIVERATHERFROM
output: OWFICERTAMLEPOSITIOERATHERTROC 

source: NXHUUVRCDTOQYHXDUWLVCKJWVXXM
actual: AWAYBILLJOBWHATREDUCELAYTEND
output: ARACLILLROBOHATREDUCELASTEED 

source: UBXYEAHDCVRYUOQTUOOFJKOZBX
actual: STRUCTUREAUDIENCEKNOWLEDGE
output: STRUCTUREAUDIENCENNOWLEDCE 

source: HGVKYFWDIHMOCHHIWLMH
ac

source: OAXGTWCPHFTRBKCYHICJYRSXDYJKS
actual: PERFORMANCEBEYONDFEWMUCHMOUTH
output: PERFORMANCEBEYONDFEWNUCHMOATY 

source: NPHAATPCCBBVAOSOYFSGDUOVQIOTVM
actual: AVAILABLEWONDERANIMALREPORTKID
output: AOAILABLEWONDERANIMALREPORTOND 

source: GFLRCFVYCQVGVEXVUGPIFHEEWRWPKSAH
actual: HISBESTNEEDANOTHERRESPONSIBILITY
output: RISMESTNEEDANOTHERRESPONMISITITE 

source: AFUKUWWPPNXHUVKDTFCJOPPMAPLQOZ
actual: NICEBREAKSIMILARVIEWTHROUGHOUT
output: EICEBREALSIMILARCIEWTHROUIROUT 

source: HFEKLUAYFYKVDJJTUAKSHK
actual: GIVEALONGMANAGECENTRAL
output: NIAEALONGTANAGECENTRAL 

source: OZSHCIXRCSMOCZVCDNOFOPOJIROEFQZVBS
actual: PULLEVIDENCETRUTHANOTHEREITHERLOOK
output: PULLEVIDENCETRUTHANOTHEREITGERTINY 

source: KAWAYKWDQPYZIZJ
actual: BEDINNERALLSURE
output: BEDINNERALLSURE 

source: GEPCCKVGZQUOJDCYHLMEXYRAUBG
actual: HANDENTIREREGIONDESIGNPLACE
output: RANDENTIREREGIONDESITNPLACE 

source: NGVICIWDTWBYHMHOYNUUTQXVXYIHYZB
actual: AREAEVERYBODYPLANALMOSTPROPERTY
output: AREAEVERYMODYPAANALLOST

source: JEUORTNZQFEXFXVGONTI
actual: FACTMAYFACTISSUEMAKE
output: FACTMAYFACTISSUEMAHE 

source: YEGSLURWNTGBBMVMSFEGIKEY
actual: LAYWALLSHOWREPUBLICANLOW
output: LAYWALLSHOAREPUOLICANLON 

source: RZXAYDVOZQTGJOQTNWYOSCSFKK
actual: DURINGTHREEAGENCYDIFFICULT
output: DURINGTHREEAGENCYDITFICULT 

source: YAHCWMADYCKUCEILFMCSHVJGHJRQMA
actual: LEADSPORTFACTOFFICERAGAINAMONG
output: LEADSPORTEACTOFFICERANAINALONT 

source: RFXKEAXHHOBVUJLCJLVQCIXBIJR
actual: DIRECTIONTONIGHTREFLECTTEAM
output: DIRECTIONTONIGHTREPLECTTEAL 



In [17]:
def score(predicted_plain, correct_plain):
    correct = 0

    for p, c in zip(predicted_plain, correct_plain):
        if str_score(p, c) > 0.8:
            correct += 1

    return correct / len(correct_plain)

In [18]:
def str_score(str_a, str_b):
    if len(str_a) != len(str_b):
        return 0

    n_correct = 0

    for a, b in zip(str_a, str_b):
        n_correct += int(a == b)

    return n_correct / len(str_a)

In [19]:
score(predicts, test_plain)

0.889

In [20]:
score(predicts, test_plain)

0.889

In [ ]:
0.092*100